In [1]:
import os
import glob
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap as LSCm
from scipy.interpolate import griddata
from scipy.signal import savgol_filter
from scipy import integrate


In [2]:
from NesrHydrusAnalyst import *

In [3]:
src = '../Datasets/sample3d'

# Modifying data grabbing

In [4]:
# tests get_grid_values
data_frame= read_hydrus_data(folder=src, save_to_csv=False)

In [ ]:
v=0
X, Z, M, x_vals, z_vals = get_grid_values(data_frame,variable=v)

In [ ]:
# print('x_vals{}, z_vals{}, X{}, Z{}, M{}'.format(x_vals.shape, 
#                                                  z_vals.shape, X.shape, 
#                                                  Z.shape, M.shape))

In [ ]:
get_available_timesteps(data_frame)

In [ ]:
get_full_dimensions(data_frame)

In [ ]:
get_legend_range(M.min(), M.max())

# Get other data from 3D simulation

In [ ]:
get_one_line_df(src, simulation_name="Sand Ditch simulation", dims='3d').T